# Tutorial Notebook for IBoat - PMCTS

<div class="alert-danger">
*Please be aware that this tutorial shall not replace the project's thorough documentation that you can find at :
https://pbarde.github.io/IBoat-PMCTS/ * 
</div>

<div class="alert-success"> **Objectives :** <br/>
    
<nl>
After completing this tutorial you should be able to use all the tools developed during the IBoat - PMCTS projet. Using these tools you can investigate different algorithms' tunings in order to find an optimal strategy and compare it to the isochrones method. <br/>

<nl>    
** Finally, all the code of this notebook can be found in the following [Tutorial script](Tutorial.py) which will run more smoothly than in this jupyter notebook (mostly because of multiprocesses involved in the interactive plots).**
</div>

** Table of contents**

1. [Before starting](#before)
2. [Weather forecasts and Simulators](#weather)
    1. [Downloading weather forecasts](#dl)
    2. [Loading weather objects](#load)
    3. [Creating simulators and displaying wind conditions](#simu)
3. [The Parallel Monte-Carlo Tree Search](#PMCTS)
  1. [Initializing the search](#init)
  2. [Creating a Forest and launching a PMCTS](#forest)
  3. [Processing and saving results](#process)
4. [Isochrones and Validation](#iso)

## <a id="before"></a>1. Before starting

<div class="alert-danger">
    Why don't you create a **conda environment** to make sure everything plays smoothly ? <br/>
Install Anaconda 3 ([Anaconda Download](https://www.anaconda.com/download/#linux)) if not done already and run the following commands in your terminal (linux of course).
</div>
```
conda create --name mcts
source activate mcts
conda install pip
conda install jupyter
conda install basemap
conda install netcdf4
conda install scipy
```
*If you don't have TkAgg and Qt5Agg you'll need to install them with:*
```
sudo apt-get install python3-tk
sudo apt-get install python3-qt
```

*Then, go to the [Tutorial notebook](Tutorial.ipynb)'s directory and run:*
```
jupyter notebook "Tutorial.ipynb"
```

*Finally, when you are finished playing you can just remove the conda environment with the lines:*
```
source deactivate
conda remove --name mcts --all
```

<div class="alert-danger"> 
    ** /!\ Please do run the next cell once: ** 
    <div/>

In [1]:
import os
os.chdir(os.getcwd() + '/solver/') # just to make sure you're working in the proper directory

## <a id="weather"></a> 2. Weather forecasts and Simulators

In this section you'll learn how to download, load, process and visualize weather forecasts. You will also create the simulators that are used by the workers in the PMCTS.

### <a id="dl"></a> A. Downloading weather forecasts

Let's start by downloading some forecasts. 
<div class="alert-warning">
*You might want to change the* `mydate` *variable. Note that if it's not working you might have chosen a date whose weather forecasts are not available anymore on the server or your proxy doesn't allow you to download with the* `netcdf4` *package. Or, you just got the date format wrong.* </div>

In [2]:
import forest as ft
import time

# The starting day of the forecast. If it's too ancient, the forecast might not be available anymore
mydate = time.strftime("%Y%m%d")  # mydate = '20180228' # for February 2, 2018

# We will download the mean scenario (id=0) and the first 2 perturbed scenarios
scenario_ids = range(3)
ft.download_scenarios(mydate, latBound=[40, 50], lonBound=[-15 + 360, 360], scenario_ids=scenario_ids)

Saved into : ../data/20180302_0000z.obj
Saved into : ../data/20180302_0100z.obj
Saved into : ../data/20180302_0200z.obj


### <a id="load"></a>B. Loading weather objects


Now that we have downloaded some forecasts, let's load them in order to create simulators with them.  

In [2]:
import forest as ft
import time

# The starting day of the forecast. If it's too ancient, the forecast might not be available anymore
mydate = time.strftime("%Y%m%d")  # mydate = '20180228' # for February 2, 2018

# We will download the mean scenario (id=0) and the first 2 perturbed scenarios
scenario_ids = range(3)
Weathers = ft.load_scenarios(mydate, latBound=[40, 50], lonBound=[-15 + 360, 360], scenario_ids=scenario_ids)

Loaded : ../data/20180302_0000z.obj
Loaded : ../data/20180302_0100z.obj
Loaded : ../data/20180302_0200z.obj


### <a id="simu"></a> C. Creating simulators and displaying wind conditions

We define the main parameters of our simulators, we create the simulators and we visualize their wind conditions. The plots are interactives (use the upper-left icons to navigate through the weather forecast).
<div class="alert-warning">
*Our code is not supposed to be executed in a jupyter notebook which is a bit capricious and does not handle mutli-processing for animations properly. So you can only animate one scenario at a time. To visualize multiple scenarios simulataneously from you should use* `ft.play_multiple_scenarios(Sims)` *in a script.*
</div>

In [3]:
%%capture
% matplotlib qt
import matplotlib.pyplot as plt
from multiprocessing import Process
from simulatorTLKT import Boat

Boat.UNCERTAINTY_COEFF = 0.02 # Characterizes the uncertainty on the boat's dynamics
NUMBER_OF_SIM = 3  # <=20 
SIM_TIME_STEP = 6  # in hours
STATE_INIT = [0, 44, 355]
N_DAYS_SIM = 3  # time horizon in days

Sims = ft.create_simulators(Weathers, numberofsim=NUMBER_OF_SIM, simtimestep=SIM_TIME_STEP,
                            stateinit=STATE_INIT, ndaysim=N_DAYS_SIM)        

# in the notebook we can visualize scenarios only one by one. 
Sims[0].play_scenario()

## /!\ if executing from a .py script, you better use this to have multiple interactive plots: 
#ft.play_multiple_scenarios(Sims)

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))



*If you feel frustrated by the interactive plot you are invited to copy paste the code below in a .py file and replace* 
```python
for ii, sim in enumerate(Sims) : 
    sim.play_scenario(ii)
```
*by*
```python
ft.play_multiple_scenarios(Sims)
```
*execute your new .py file and enjoy.<br/>
(by the way, if you are lazy, you can also give the [Tutorial script](Tutorial.py) a run)*


## <a id="PMCTS"></a> 3. The Parallel Monte-Carlo Tree Search

In this section we will see how to launch a PMCTS, process and visualize the results.

### <a id="init"></a> A. Initializing the search

First of all we define a departure point, a mission heading and we compute the corresponding destination point and reference travel time. Then, we visualize the mean trajectories per scenario during the two initialization phases. 

In [11]:
%%capture
% matplotlib qt
import matplotlib
matplotlib.rcParams.update({'font.size': 10})

missionheading = 0 # direction wrt. true North we want to go the furthest.
ntra = 50 # number of trajectories used during the initialization
destination, timemin = ft.initialize_simulators(Sims, ntra, STATE_INIT, missionheading, plot=True)

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [6]:
print("destination : {} & timemin : {}".format(destination, timemin))

destination : [47.475489975682045, 355.0] & timemin : 2.6654691601516185


### <a id="forest"></a> B. Creating a Forest and launching a PMCTS

Now we create a Forest (the object managing the worker trees and the master tree) and we launch a search. To do this we must define the exploration parameters of the search.

In [7]:
import worker

##Exploration Parameters##
worker.RHO = 0.5 #Exploration coefficient in the UCT formula.
worker.UCT_COEFF = 1 / 2 ** 0.5 #Proportion between master utility and worker utility of node utility.

budget = 100 # number of nodes we want to expand in each worker
frequency = 10 # number of steps performed by a worker before writing the results into the master
forest = ft.Forest(listsimulators=Sims, destination=destination, timemin=timemin, budget=budget)

if __name__ == '__main__':
    master_nodes = forest.launch_search(STATE_INIT, frequency)


Iteration 50 on 100 for workers 1

Iteration 50 on 100 for workers 2

Iteration 50 on 100 for workers 0

Iteration 100 on 100 for workers 1

Iteration 100 on 100 for workers 2

Iteration 100 on 100 for workers 0



Since the `master_nodes` object was created as a memory shared by multiple processes we need to do a deep copy of it before processing it. 

In [8]:
from master_node import deepcopy_dict
new_dict = deepcopy_dict(master_nodes)

### <a id="save"></a> C. Processing and saving results

At this point we can create a `MasterTree` object to process the results and get the optimal policies. We usually add it to the forest object. 

In [9]:
from master import MasterTree
forest.master = MasterTree(Sims, destination, nodes=new_dict)
forest.master.get_best_policy()

Global policy
Depth 1: best reward = 0.3333333333333333 for action = 0
Depth 2: best reward = 0.4375 for action = 180
Depth 3: best reward = 0.6041666666666666 for action = 270
Policy for scenario 0
Depth 1: best reward = 0.5625 for action = 0
Depth 2: best reward = 0.8125 for action = 45
Depth 3: best reward = 0.8125 for action = 90
Policy for scenario 1
Depth 1: best reward = 0.3125 for action = 0
Depth 2: best reward = 0.6875 for action = 180
Depth 3: best reward = 0.6875 for action = 225
Policy for scenario 2
Depth 1: best reward = 0.375 for action = 0
Depth 2: best reward = 0.5625 for action = 225
Depth 3: best reward = 0.1875 for action = 225


It is also possible to plot the master (or the worker corresponding to scenario 1) tree colored with nodes colored by utility, exploration and exploitation values.

In [12]:
%%capture
% matplotlib tk
forest.master.plot_tree_uct();
forest.master.plot_tree_uct(1);

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


We can also follow the global best policy and look at the reward distribution along this path.

In [14]:
%%capture
% matplotlib qt
forest.master.plot_hist_best_policy(interactive = True)

/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/font_manager.py:1320: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
Traceback (most recent call last):
  File "/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/cbook/__init__.py", line 388, in process
    proxy(*args, **kwargs)
  File "/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/cbook/__init__.py", line 228, in __call__
    return mtd(*args, **kwargs)
  File "/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/matplotlib/animation.py", line 1085, in _start
    self.event_source.add_callback(self._step)
AttributeError: 'NoneType' object has no attribute 'add_callback'


Finally we can save the results. 

In [15]:
forest.master.save_tree("my_tuto_results")

## <a id="iso"></a> 4. Isochrones and Validation

In this section we will see how to run a isochrone search and compare its performances.

### <a id="iso_search"></a> A. Setting up the search

To set up a isochrones search we need the mean scenario and a simulator without noise on the boat dynamics.

In [19]:
% matplotlib qt
import sys
sys.path.append('../isochrones/')
sys.path.append('../model/')
import IsochroneClass as IC
import simulatorTLKT as SimC

SimC.Boat.UNCERTAINTY_COEFF = 0
sim = Sims[0]

solver_iso = IC.Isochrone(sim, STATE_INIT[1:3], destination, delta_cap=10, increment_cap=9, nb_secteur=200,
                          resolution=300)

temps_estime, plan_iso, plan_iso_ligne_droite, trajectoire = solver_iso.isochrone_methode()

m = sim.prepareBaseMap(centerOfMap=STATE_INIT[1:], proj='aeqd')
for i, el in enumerate(trajectoire):
    trajectoire[i] = [i] + el

sim.plotTraj(trajectoire, m, quiv=True, scatter=False)

##########################################################

SimC.Boat.UNCERTAINTY_COEFF = 0.2

IC.comparateur_plans_optimaux(Sims,ntra,STATE_INIT,destination,plan_iso,my_tuto_results,True)

Pas de solution trouvée dans le temps imparti.
Veuillez raffiner vous paramètres de recherche.


/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()
/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:1623: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  fill_color = ax.get_axis_bgcolor()
/home/paul/anaconda3/envs/mcts/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:1698: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  limb = ax.axesPatch


TypeError: 'NoneType' object is not iterable

In [20]:
print(STATE_INIT[1:3])
print(destination)

[44, 355]
[47.12836169801304, 355.0]
